In [28]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../fraud_detection/src/")
from util import s_to_time_format, string_to_datetime,hour_to_range
from tqdm import tqdm

def value_to_count(df_train, df_test, df_train_normal_cano_id, df_):
    # separate continuous feature and categorial features
    feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
       'loctm_minute_of_hour', 'loctm_second_of_min'] 
    cont_feats = ['iterm', 
                  'locdt',
                  'loctm_hour_of_day',
                  'loctm_minute_of_hour', 
                  'loctm_second_of_min']
    feats = [f for f in feats if f not in cont_feats]
    # we only coner categorial features
    
    df = pd.concat([df_train[feats], df_test[feats]], axis = 0)
    for f in tqdm(feats):
        count_dict = df[f].value_counts(dropna = False).to_dict() 
        df_train_normal_cano_id[f] = df_train_normal_cano_id[f].apply(lambda v: count_dict[v])
        df_train[f] = df_train[f].apply(lambda v: count_dict[v])
        df_test[f] = df_test[f].apply(lambda v: count_dict[v])
        df_[f] = df_[f].apply(lambda v: count_dict[v])
    return df_train,df_test,df_train_normal_cano_id, df_

def feature_normalization_auto(df_train, df_test, df_train_normal_cano_id,df_):
    """
    return two inputs of autoencoder, one is for train and another one is for test
    """
    #from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
    feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
       'loctm_minute_of_hour', 'loctm_second_of_min']
    df = pd.concat([df_train[feats], df_test[feats]], axis = 0)


    for f in tqdm(feats):
        try:
            #scaler = MinMaxScaler()
            max_ = df[f].max()
            min_ = df[f].min()
            df_train_normal_cano_id[f] = df_train_normal_cano_id[f].apply(lambda x: (x-min_)/(max_-min_))
            df_[f] = df_[f].apply(lambda x: (x-min_)/(max_-min_))
            #df_test[f] = df_test[f].apply(lambda x: (x-min_)/(max_-min_))
        except:
            print(f)
    return df_train_normal_cano_id,df_

def partition_(df, num_features):
    data = []
    for i in range(len(df)):
        out = None
        if i == 0:
            out = np.concatenate(((np.zeros((2,num_features))),df.iloc[:1].values))
        elif i== 1:
            out = np.concatenate(((np.zeros((1,num_features))),df.iloc[:i+1].values))
        else:
            out = df.iloc[i+1-3:i+1].values
        data.append(out)
    return data

def partition(df_, sequence_length = 3):
    feats = [f for f in df_.columns if f not in {"fraud_ind","cano_help","locdt_help"}]
    sequences = []
    for _, df in df_.groupby(by = "cano_help"):
        data = partition_(df[feats], num_features = len(feats))
        for d in data:
            sequences.append(d)
    return sequences

def get_sequence_dataframe(df):
    df_train_sequences = partition(df)
    df_train_sequences = np.concatenate(df_train_sequences)
    df_train_sequences = pd.DataFrame(df_train_sequences)
    return df_train_sequences


#-----------------------------
# load data
#-----------------------------
df_train = pd.read_csv("/data/yunrui_li/fraud/dataset/train.csv")
df_test = pd.read_csv("/data/yunrui_li/fraud/dataset/test.csv")


for df in [df_train, df_test]:
    # pre-processing
    df["loctm_"] = df.loctm.astype(int).astype(str)
    df.loctm_ = df.loctm_.apply(s_to_time_format).apply(string_to_datetime)
    # time-related feature
    df["loctm_hour_of_day"] = df.loctm_.apply(lambda x: x.hour)
    df["loctm_minute_of_hour"] = df.loctm_.apply(lambda x: x.minute)
    df["loctm_second_of_min"] = df.loctm_.apply(lambda x: x.second)

    # removed the columns no need
    df.drop(columns = ["loctm_", "loctm","txkey"], axis = 1, inplace = True)

df_train["cano_locdt_index"] = ["{}_{}_{}_{}_{}".format(str(i),str(j),str(k),str(l),str(m)) for i,j,k,l,m in zip(df_train.cano,
                                                                                   df_train.locdt,
                                                                                   df_train.loctm_hour_of_day,
                                                                                   df_train.loctm_minute_of_hour,
                                                                                   df_train.loctm_second_of_min,
                                                                                  )]
df_test["cano_locdt_index"] = ["{}_{}_{}_{}_{}".format(str(i),str(j),str(k),str(l),str(m)) for i,j,k,l,m in zip(df_test.cano,
                                                                                  df_test.locdt,
                                                                                  df_train.loctm_hour_of_day,
                                                                                  df_train.loctm_minute_of_hour,
                                                                                  df_train.loctm_second_of_min,
                                                                                 )]

df_train["cano_help"] = df_train.cano
df_test["cano_help"] = df_test.cano

df_train["locdt_help"] = df_train.locdt
df_test["locdt_help"] = df_test.locdt

df_train["loctm_hour_of_day_help"] = df_train.loctm_hour_of_day
df_test["loctm_hour_of_day_help"] = df_test.loctm_hour_of_day

df_train["loctm_minute_of_hour_help"] = df_train.loctm_minute_of_hour
df_test["loctm_minute_of_hour_help"] = df_test.loctm_minute_of_hour

df_train["loctm_second_of_min_help"] = df_train.loctm_second_of_min
df_test["loctm_second_of_min_help"] = df_test.loctm_second_of_min

#-----------------------------
# feature extraction
#-----------------------------
df = pd.concat([df_train, df_test], axis = 0)
df.sort_values(by = ["cano", "locdt","loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"], inplace = True)

#-----------------------------
# prepare training data
#-----------------------------
df_train.sort_values(by = ["cano", "locdt","loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"], inplace = True)

# df_train, df_test = value_to_count(df_train, df_test)
# df_train, df_test = feature_normalization_auto(df_train, df_test)

fraud_cano_id = df_train[df_train.fraud_ind == 1].cano.unique().tolist()

df_train_normal_cano_id = df_train[~df_train.cano.isin(fraud_cano_id)]
print ("number of training data",df_train_normal_cano_id.shape)

df_train, df_test, df_train_normal_cano_id, df = value_to_count(df_train, df_test,df_train_normal_cano_id, df)
df_train_normal_cano_id, df = feature_normalization_auto(df_train, df_test,df_train_normal_cano_id, df)

#-----------------------------
# post-processing
#-----------------------------
df.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
df_train_normal_cano_id.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
   'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
   'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
   'loctm_minute_of_hour', 'loctm_second_of_min'] + ["cano_locdt_index","cano_help","locdt_help",
#                                                      "loctm_hour_of_day_help",
#                                                      "loctm_minute_of_hour_help",
#                                                      "loctm_second_of_min_help",
                                                    ]

df = df[feats]
df_train_normal_cano_id = df_train_normal_cano_id[feats]

#-----------------------------
# get train/test data
#-----------------------------

X_train = get_sequence_dataframe(df_train_normal_cano_id)
Feature = get_sequence_dataframe(df)
#-----------------------------
# modeling (unsupervised learning)
#-----------------------------

KeyboardInterrupt: 

In [2]:
X_train.shape

(4171146, 24)

In [3]:
Feature.shape

(5830356, 24)

In [4]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.245115,0.00716846,0.00716846,0.0142658,1,1,1,0.783407,1,1,...,1,0.0771681,1,1,1,1,0.652174,0.322034,0.813559,0_1_15_19_48
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.245115,0.00716846,0.00716846,0.0142658,1,1,1,0.783407,1,1,...,1,0.0771681,1,1,1,1,0.652174,0.322034,0.813559,0_1_15_19_48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171141,0.0418379,0.00268817,0.00268817,0.00118336,1,0.0102579,1,1,0.00517195,0,...,0.0573014,0.000470825,1,0.0044661,1,1,0.565217,0.627119,0.372881,213334_90_13_37_22
4171142,0.365315,0.00268817,0.00268817,1,1,0.0102579,1,1,0.00517195,0,...,0.568704,0.000674849,1,1,1,1,0.652174,0.779661,0.711864,213334_90_15_46_42
4171143,0.0418379,0.00268817,0.00268817,0.00118336,1,0.0102579,1,1,0.00517195,0,...,0.0573014,0.000470825,1,0.0044661,1,1,0.565217,0.627119,0.372881,213334_90_13_37_22
4171144,0.365315,0.00268817,0.00268817,1,1,0.0102579,1,1,0.00517195,0,...,0.568704,0.000674849,1,1,1,1,0.652174,0.779661,0.711864,213334_90_15_46_42


In [5]:
Feature

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.245115,0.00716846,0.00716846,0.0142658,1,1,1,0.783407,1,1,...,1,0.0771681,1,1,1,1,0.652174,0.322034,0.813559,0_1_15_19_48
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.245115,0.00716846,0.00716846,0.0142658,1,1,1,0.783407,1,1,...,1,0.0771681,1,1,1,1,0.652174,0.322034,0.813559,0_1_15_19_48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830351,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5830352,1,0,0,5.94654e-06,0.0533723,1,0,0.955971,1,1,...,0.328259,0.0398945,1,1,1,1,0.608696,0.237288,0.135593,213572_120_18_49_52
5830353,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5830354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import trange
import sys
sys.path.append("../DeepADoTS/src/algorithms/")
# from .algorithm_utils import Algorithm, PyTorchUtils
# from .autoencoder import AutoEncoderModule
#from lstm_enc_dec_axl import LSTMEDModule
import abc
import logging
import random

import numpy as np
import torch
import tensorflow as tf
from tensorflow.python.client import device_lib
from torch.autograd import Variable


class Algorithm(metaclass=abc.ABCMeta):
    def __init__(self, module_name, name, seed, details=False):
        self.logger = logging.getLogger(module_name)
        self.name = name
        self.seed = seed
        self.details = details
        self.prediction_details = {}

        if self.seed is not None:
            random.seed(seed)
            np.random.seed(seed)

    def __str__(self):
        return self.name

    @abc.abstractmethod
    def fit(self, X):
        """
        Train the algorithm on the given dataset
        """

    @abc.abstractmethod
    def predict(self, X):
        """
        :return anomaly score
        """


class PyTorchUtils(metaclass=abc.ABCMeta):
    def __init__(self, seed, gpu):
        self.gpu = gpu
        self.seed = seed
        if self.seed is not None:
            torch.manual_seed(self.seed)
            torch.cuda.manual_seed(self.seed)
        self.framework = 0

    @property
    def device(self):
        #return 'cuda:0'
        return torch.device(f'cuda:{self.gpu}' if torch.cuda.is_available() and self.gpu is not None else 'cpu')

    def to_var(self, t, **kwargs):
        # ToDo: check whether cuda Variable.
        t = t.to(self.device)
        return Variable(t, **kwargs)

    def to_device(self, model):
        model.to(self.device)


class TensorflowUtils(metaclass=abc.ABCMeta):
    def __init__(self, seed, gpu):
        self.gpu = gpu
        self.seed = seed
        if self.seed is not None:
            tf.set_random_seed(seed)
        self.framework = 1

    @property
    def device(self):
        local_device_protos = device_lib.list_local_devices()
        gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
        return tf.device(gpus[self.gpu] if gpus and self.gpu is not None else '/cpu:0')
    
import logging

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import multivariate_normal
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import trange


class AutoEncoder(Algorithm, PyTorchUtils):
    def __init__(self, name: str='AutoEncoder', num_epochs: int=10, batch_size: int=20, lr: float=1e-3,
                 hidden_size: int=5, sequence_length: int=30, train_gaussian_percentage: float=0.25,
                 seed: int=None, gpu: int=None, details=True):
        Algorithm.__init__(self, __name__, name, seed, details=details)
        PyTorchUtils.__init__(self, seed, gpu)
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lr = lr

        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.train_gaussian_percentage = train_gaussian_percentage

        self.aed = None
        self.mean, self.cov = None, None

    def fit(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        indices = np.random.permutation(len(sequences))
        split_point = int(self.train_gaussian_percentage * len(sequences))
        train_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                  sampler=SubsetRandomSampler(indices[:-split_point]), pin_memory=True)
        train_gaussian_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                           sampler=SubsetRandomSampler(indices[-split_point:]), pin_memory=True)

        self.aed = AutoEncoderModule(X.shape[1], self.sequence_length, self.hidden_size, seed=self.seed, gpu=self.gpu)
        self.to_device(self.aed)  # .double()
        optimizer = torch.optim.Adam(self.aed.parameters(), lr=self.lr)

        self.aed.train()
        for epoch in trange(self.num_epochs):
            logging.debug(f'Epoch {epoch+1}/{self.num_epochs}.')
            for ts_batch in train_loader:
                output = self.aed(self.to_var(ts_batch))
                loss = nn.MSELoss(size_average=False)(output, self.to_var(ts_batch.float()))
                self.aed.zero_grad()
                loss.backward()
                optimizer.step()

        self.aed.eval()
        error_vectors = []
        for ts_batch in train_gaussian_loader:
            output = self.aed(self.to_var(ts_batch))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts_batch.float()))
            error_vectors += list(error.view(-1, X.shape[1]).data.cpu().numpy())

        self.mean = np.mean(error_vectors, axis=0)
        self.cov = np.cov(error_vectors, rowvar=False)

    def predict(self, X: pd.DataFrame) -> np.array:
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, shuffle=False, drop_last=False)

        self.aed.eval()
        mvnormal = multivariate_normal(self.mean, self.cov, allow_singular=True)
        scores = []
        outputs = []
        errors = []
        for idx, ts in enumerate(data_loader):
            output = self.aed(self.to_var(ts))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts.float()))
            score = -mvnormal.logpdf(error.view(-1, X.shape[1]).data.cpu().numpy())
            scores.append(score.reshape(ts.size(0), self.sequence_length))
            if self.details:
                outputs.append(output.data.numpy())
                errors.append(error.data.numpy())

        # stores seq_len-many scores per timestamp and averages them
        scores = np.concatenate(scores)
        lattice = np.full((self.sequence_length, X.shape[0]), np.nan)
        for i, score in enumerate(scores):
            lattice[i % self.sequence_length, i:i + self.sequence_length] = score
        scores = np.nanmean(lattice, axis=0)

        if self.details:
            outputs = np.concatenate(outputs)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, output in enumerate(outputs):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = output
            self.prediction_details.update({'reconstructions_mean': np.nanmean(lattice, axis=0).T})

            errors = np.concatenate(errors)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, error in enumerate(errors):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = error
            self.prediction_details.update({'errors_mean': np.nanmean(lattice, axis=0).T})

        return scores


class AutoEncoderModule(nn.Module, PyTorchUtils):
    def __init__(self, n_features: int, sequence_length: int, hidden_size: int, seed: int, gpu: int):
        # Each point is a flattened window and thus has as many features as sequence_length * features
        super().__init__()
        PyTorchUtils.__init__(self, seed, gpu)
        input_length = n_features * sequence_length

        # creates powers of two between eight and the next smaller power from the input_length
        dec_steps = 2 ** np.arange(max(np.ceil(np.log2(hidden_size)), 2), np.log2(input_length))[1:]
        dec_setup = np.concatenate([[hidden_size], dec_steps.repeat(2), [input_length]])
        enc_setup = dec_setup[::-1]

        layers = np.array([[nn.Linear(int(a), int(b)), nn.Tanh()] for a, b in enc_setup.reshape(-1, 2)]).flatten()[:-1]
        self._encoder = nn.Sequential(*layers)
        self.to_device(self._encoder)

        layers = np.array([[nn.Linear(int(a), int(b)), nn.Tanh()] for a, b in dec_setup.reshape(-1, 2)]).flatten()[:-1]
        self._decoder = nn.Sequential(*layers)
        self.to_device(self._decoder)

    def forward(self, ts_batch, return_latent: bool=False):
        flattened_sequence = ts_batch.view(ts_batch.size(0), -1)
        enc = self._encoder(flattened_sequence.float())
        dec = self._decoder(enc)
        reconstructed_sequence = dec.view(ts_batch.size())
        return (reconstructed_sequence, enc) if return_latent else reconstructed_sequence
    
import logging

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import multivariate_normal
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import trange

# from .algorithm_utils import Algorithm, PyTorchUtils


class LSTMED(Algorithm, PyTorchUtils):
    def __init__(self, name: str='LSTM-ED', num_epochs: int=10, batch_size: int=20, lr: float=1e-3,
                 hidden_size: int=5, sequence_length: int=30, train_gaussian_percentage: float=0.25,
                 n_layers: tuple=(1, 1), use_bias: tuple=(True, True), dropout: tuple=(0, 0),
                 seed: int=None, gpu: int = None, details=True):
        Algorithm.__init__(self, __name__, name, seed, details=details)
        PyTorchUtils.__init__(self, seed, gpu)
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lr = lr

        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.train_gaussian_percentage = train_gaussian_percentage

        self.n_layers = n_layers
        self.use_bias = use_bias
        self.dropout = dropout

        self.lstmed = None
        self.mean, self.cov = None, None

    def fit(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        indices = np.random.permutation(len(sequences))
        split_point = int(self.train_gaussian_percentage * len(sequences))
        train_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                  sampler=SubsetRandomSampler(indices[:-split_point]), pin_memory=True)
        train_gaussian_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                           sampler=SubsetRandomSampler(indices[-split_point:]), pin_memory=True)

        self.lstmed = LSTMEDModule(X.shape[1], self.hidden_size,
                                   self.n_layers, self.use_bias, self.dropout,
                                   seed=self.seed, gpu=self.gpu)
        self.to_device(self.lstmed)
        optimizer = torch.optim.Adam(self.lstmed.parameters(), lr=self.lr)

        self.lstmed.train()
        for epoch in trange(self.num_epochs):
            logging.debug(f'Epoch {epoch+1}/{self.num_epochs}.')
            for ts_batch in train_loader:
                output = self.lstmed(self.to_var(ts_batch))
                loss = nn.MSELoss(size_average=False)(output, self.to_var(ts_batch.float()))
                self.lstmed.zero_grad()
                loss.backward()
                optimizer.step()

        self.lstmed.eval()
        error_vectors = []
        for ts_batch in train_gaussian_loader:
            output = self.lstmed(self.to_var(ts_batch))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts_batch.float()))
            error_vectors += list(error.view(-1, X.shape[1]).data.cpu().numpy())

        self.mean = np.mean(error_vectors, axis=0)
        self.cov = np.cov(error_vectors, rowvar=False)

    def predict(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, shuffle=False, drop_last=False)

        self.lstmed.eval()
        mvnormal = multivariate_normal(self.mean, self.cov, allow_singular=True)
        scores = []
        outputs = []
        errors = []
        for idx, ts in enumerate(data_loader):
            output = self.lstmed(self.to_var(ts))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts.float()))
            score = -mvnormal.logpdf(error.view(-1, X.shape[1]).data.cpu().numpy())
            scores.append(score.reshape(ts.size(0), self.sequence_length))
            if self.details:
                outputs.append(output.data.numpy())
                errors.append(error.data.numpy())

        # stores seq_len-many scores per timestamp and averages them
        scores = np.concatenate(scores)
        lattice = np.full((self.sequence_length, data.shape[0]), np.nan)
        for i, score in enumerate(scores):
            lattice[i % self.sequence_length, i:i + self.sequence_length] = score
        scores = np.nanmean(lattice, axis=0)

        if self.details:
            outputs = np.concatenate(outputs)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, output in enumerate(outputs):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = output
            self.prediction_details.update({'reconstructions_mean': np.nanmean(lattice, axis=0).T})

            errors = np.concatenate(errors)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, error in enumerate(errors):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = error
            self.prediction_details.update({'errors_mean': np.nanmean(lattice, axis=0).T})

        return scores


class LSTMEDModule(nn.Module, PyTorchUtils):
    def __init__(self, n_features: int, hidden_size: int,
                 n_layers: tuple, use_bias: tuple, dropout: tuple,
                 seed: int, gpu: int):
        super().__init__()
        PyTorchUtils.__init__(self, seed, gpu)
        self.n_features = n_features
        self.hidden_size = hidden_size

        self.n_layers = n_layers
        self.use_bias = use_bias
        self.dropout = dropout

        self.encoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True,
                               num_layers=self.n_layers[0], bias=self.use_bias[0], dropout=self.dropout[0])
        self.to_device(self.encoder)
        self.decoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True,
                               num_layers=self.n_layers[1], bias=self.use_bias[1], dropout=self.dropout[1])
        self.to_device(self.decoder)
        self.hidden2output = nn.Linear(self.hidden_size, self.n_features)
        self.to_device(self.hidden2output)

    def _init_hidden(self, batch_size):
        return (self.to_var(torch.Tensor(self.n_layers[0], batch_size, self.hidden_size).zero_()),
                self.to_var(torch.Tensor(self.n_layers[0], batch_size, self.hidden_size).zero_()))

    def forward(self, ts_batch, return_latent: bool=False):
        batch_size = ts_batch.shape[0]

        # 1. Encode the timeseries to make use of the last hidden state.
        enc_hidden = self._init_hidden(batch_size)  # initialization with zero
        _, enc_hidden = self.encoder(ts_batch.float(), enc_hidden)  # .float() here or .double() for the model

        # 2. Use hidden state as initialization for our Decoder-LSTM
        dec_hidden = enc_hidden

        # 3. Also, use this hidden state to get the first output aka the last point of the reconstructed timeseries
        # 4. Reconstruct timeseries backwards
        #    * Use true data for training decoder
        #    * Use hidden2output for prediction
        output = self.to_var(torch.Tensor(ts_batch.size()).zero_())
        for i in reversed(range(ts_batch.shape[1])):
            output[:, i, :] = self.hidden2output(dec_hidden[0][0, :])

            if self.training:
                _, dec_hidden = self.decoder(ts_batch[:, i].unsqueeze(1).float(), dec_hidden)
            else:
                _, dec_hidden = self.decoder(output[:, i].unsqueeze(1), dec_hidden)

        return (output, enc_hidden[1][-1]) if return_latent else output

"""Adapted from Daniel Stanley Tan (https://github.com/danieltan07/dagmm)"""
import logging
import sys
sys.path.append("../DeepADoTS/src/algorithms/")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import trange

# from .algorithm_utils import Algorithm, PyTorchUtils
# from .autoencoder import AutoEncoderModule
#from lstm_enc_dec_axl import LSTMEDModule


class DAGMM(Algorithm, PyTorchUtils):
    class AutoEncoder:
        NN = AutoEncoderModule
        LSTM = LSTMEDModule

    def __init__(self, num_epochs=10, lambda_energy=0.1, lambda_cov_diag=0.005, lr=1e-3, batch_size=50, gmm_k=3,
                 normal_percentile=80, sequence_length=30, autoencoder_type=AutoEncoderModule, autoencoder_args=None,
                 hidden_size: int=5, seed: int=None, gpu: int=None, details=True):
        _name = 'LSTM-DAGMM' if autoencoder_type == LSTMEDModule else 'DAGMM'
        Algorithm.__init__(self, __name__, _name, seed, details=details)
        PyTorchUtils.__init__(self, seed, gpu)
        self.num_epochs = num_epochs
        self.lambda_energy = lambda_energy
        self.lambda_cov_diag = lambda_cov_diag
        self.lr = lr
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.gmm_k = gmm_k  # Number of Gaussian mixtures
        self.normal_percentile = normal_percentile  # Up to which percentile data should be considered normal
        self.autoencoder_type = autoencoder_type
        if autoencoder_type == AutoEncoderModule:
            self.autoencoder_args = ({'sequence_length': self.sequence_length})
        elif autoencoder_type == LSTMEDModule:
            self.autoencoder_args = ({'n_layers': (1, 1), 'use_bias': (True, True), 'dropout': (0.0, 0.0)})
        self.autoencoder_args.update({'seed': seed, 'gpu': gpu})
        if autoencoder_args is not None:
            self.autoencoder_args.update(autoencoder_args)
        self.hidden_size = hidden_size

        self.dagmm, self.optimizer, self.train_energy, self._threshold = None, None, None, None

    def reset_grad(self):
        self.dagmm.zero_grad()

    def dagmm_step(self, input_data):
        self.dagmm.train()
        enc, dec, z, gamma = self.dagmm(input_data)
        #print (enc, dec, z, gamma)
        total_loss, sample_energy, recon_error, cov_diag = self.dagmm.loss_function(input_data, dec, z, gamma,
                                                                                    self.lambda_energy,
                                                                                    self.lambda_cov_diag)
        self.reset_grad()
        total_loss = torch.clamp(total_loss, max=1e7)  # Extremely high loss can cause NaN gradients
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.dagmm.parameters(), 5)
        # if np.array([np.isnan(p.grad.detach().numpy()).any() for p in self.dagmm.parameters()]).any():
        #     import IPython; IPython.embed()
        self.optimizer.step()
        return total_loss, sample_energy, recon_error, cov_diag

    def fit(self, X: pd.DataFrame):
        """Learn the mixture probability, mean and covariance for each component k.
        Store the computed energy based on the training data and the aforementioned parameters."""
        #X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(X.shape[0] - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, shuffle=True, drop_last=True)
        self.hidden_size = 5 + int(X.shape[1] / 20)
        autoencoder = self.autoencoder_type(X.shape[1], hidden_size=self.hidden_size, **self.autoencoder_args)
        self.dagmm = DAGMMModule(autoencoder, n_gmm=self.gmm_k, latent_dim=self.hidden_size + 2,
                                 seed=self.seed, gpu=self.gpu)
        self.to_device(self.dagmm)
        self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr)

        for _ in trange(self.num_epochs):
            for input_data in data_loader:
                input_data = self.to_var(input_data)
                self.dagmm_step(input_data.float())

        self.dagmm.eval()
        n = 0
        mu_sum = 0
        cov_sum = 0
        gamma_sum = 0
        for input_data in data_loader:
            input_data = self.to_var(input_data)
            _, _, z, gamma = self.dagmm(input_data.float())
            phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)

            batch_gamma_sum = torch.sum(gamma, dim=0)

            gamma_sum += batch_gamma_sum
            mu_sum += mu * batch_gamma_sum.unsqueeze(-1)  # keep sums of the numerator only
            cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1)  # keep sums of the numerator only

            n += input_data.size(0)

    def predict(self, X: pd.DataFrame):
        """Using the learned mixture probability, mean and covariance for each component k, compute the energy on the
        given data."""
        self.dagmm.eval()
        #X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(len(data) - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=1, shuffle=False)
        test_energy = np.full((self.sequence_length, X.shape[0]), np.nan)

        encodings = np.full((self.sequence_length, X.shape[0], self.hidden_size), np.nan)
        decodings = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
        euc_errors = np.full((self.sequence_length, X.shape[0]), np.nan)
        csn_errors = np.full((self.sequence_length, X.shape[0]), np.nan)

        for i, sequence in enumerate(data_loader):
            #print ("shape of sequence",self.to_var(sequence).float().shape)
            enc, dec, z, gamma = self.dagmm(self.to_var(sequence).float())
            sample_energy, _ = self.dagmm.compute_energy(z, size_average=False)
            idx = (i % self.sequence_length, np.arange(i, i + self.sequence_length))
            test_energy[idx] = sample_energy.data.numpy()

            if self.details:
                encodings[idx] = enc.data.cpu().numpy()
                decodings[idx] = dec.data.cpu().numpy()
                euc_errors[idx] = z[:, 1].data.cpu().numpy()
                csn_errors[idx] = z[:, 2].data.cpu().numpy()

        test_energy = np.nanmean(test_energy, axis=0)

        if self.details:
            self.prediction_details.update({'latent_representations': np.nanmean(encodings, axis=0).T})
            self.prediction_details.update({'reconstructions_mean': np.nanmean(decodings, axis=0).T})
            self.prediction_details.update({'euclidean_errors_mean': np.nanmean(euc_errors, axis=0)})
            self.prediction_details.update({'cosine_errors_mean': np.nanmean(csn_errors, axis=0)})

        return test_energy


class DAGMMModule(nn.Module, PyTorchUtils):
    """Residual Block."""

    def __init__(self, autoencoder, n_gmm, latent_dim, seed: int, gpu: int):
        super(DAGMMModule, self).__init__()
        PyTorchUtils.__init__(self, seed, gpu)

        self.add_module('autoencoder', autoencoder)

        layers = [
            nn.Linear(latent_dim, 10),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(10, n_gmm),
            nn.Softmax(dim=1)
        ]
        self.estimation = nn.Sequential(*layers)
        self.to_device(self.estimation)

        self.register_buffer('phi', self.to_var(torch.zeros(n_gmm)))
        self.register_buffer('mu', self.to_var(torch.zeros(n_gmm, latent_dim)))
        self.register_buffer('cov', self.to_var(torch.zeros(n_gmm, latent_dim, latent_dim)))

    def relative_euclidean_distance(self, a, b, dim=1):
        return (a - b).norm(2, dim=dim) / torch.clamp(a.norm(2, dim=dim), min=1e-10)

    def forward(self, x):
        dec, enc = self.autoencoder(x, return_latent=True)

        rec_cosine = F.cosine_similarity(x.view(x.shape[0], -1), dec.view(dec.shape[0], -1), dim=1)
        rec_euclidean = self.relative_euclidean_distance(x.view(x.shape[0], -1), dec.view(dec.shape[0], -1), dim=1)

        # Concatenate latent representation, cosine similarity and relative Euclidean distance between x and dec(enc(x))
        z = torch.cat([enc, rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)
        gamma = self.estimation(z)

        return enc, dec, z, gamma

    def compute_gmm_params(self, z, gamma):
        N = gamma.size(0)
        # K
        sum_gamma = torch.sum(gamma, dim=0)

        # K
        phi = (sum_gamma / N)

        self.phi = phi.data

        # K x D
        mu = torch.sum(gamma.unsqueeze(-1) * z.unsqueeze(1), dim=0) / sum_gamma.unsqueeze(-1)
        self.mu = mu.data
        # z = N x D
        # mu = K x D
        # gamma N x K

        # z_mu = N x K x D
        z_mu = (z.unsqueeze(1) - mu.unsqueeze(0))

        # z_mu_outer = N x K x D x D
        z_mu_outer = z_mu.unsqueeze(-1) * z_mu.unsqueeze(-2)

        # K x D x D
        cov = torch.sum(gamma.unsqueeze(-1).unsqueeze(-1) * z_mu_outer, dim=0) / sum_gamma.unsqueeze(-1).unsqueeze(-1)
        self.cov = cov.data

        return phi, mu, cov

    def compute_energy(self, z, phi=None, mu=None, cov=None, size_average=True):
        if phi is None:
            phi = Variable(self.phi)
        if mu is None:
            mu = Variable(self.mu)
        if cov is None:
            cov = Variable(self.cov)

        k, d, _ = cov.size()

        z_mu = (z.unsqueeze(1) - mu.unsqueeze(0))

        cov_inverse = []
        det_cov = []
        cov_diag = 0
        eps = 1e-12
        for i in range(k):
            # K x D x D
            cov_k = cov[i] + self.to_var(torch.eye(d) * eps)
            pinv = np.linalg.pinv(cov_k.data.cpu().numpy())
            cov_inverse.append(Variable(torch.from_numpy(pinv)).unsqueeze(0))

            eigvals = np.linalg.eigvals(cov_k.data.cpu().numpy() * (2 * np.pi))
            if np.min(eigvals) < 0:
                pass
                #logging.warning(f'Determinant was negative! Clipping Eigenvalues to 0+epsilon from {np.min(eigvals)}')
            determinant = np.prod(np.clip(eigvals, a_min=sys.float_info.epsilon, a_max=None))
            det_cov.append(determinant)

            cov_diag = cov_diag + torch.sum(1 / cov_k.diag())

        # K x D x D
        cov_inverse = torch.cat(cov_inverse, dim=0)
        # K
        det_cov = Variable(torch.from_numpy(np.float32(np.array(det_cov))))
#         print ("sum-0",cov_inverse.unsqueeze(0))
#         print ("sum-1",z_mu.ufnsqueeze(-1).cpu())
#         print ("sum", torch.sum(z_mu.unsqueeze(-1).cpu() * cov_inverse.unsqueeze(0), dim=-2))
        # N x K
        exp_term_tmp = -0.5 * torch.sum(torch.sum(z_mu.unsqueeze(-1).cpu() * cov_inverse.unsqueeze(0), dim=-2) * z_mu.cpu(), dim=-1)
        # for stability (logsumexp)
        max_val = torch.max((exp_term_tmp).clamp(min=0), dim=1, keepdim=True)[0]

        exp_term = torch.exp(exp_term_tmp - max_val)
#         print ("sample_energy", self.to_var(phi.unsqueeze(0)).cpu())
#         print ("sample_energy-exp_term", exp_term)
        sample_energy = -max_val.squeeze() - torch.log(
            torch.sum(self.to_var(phi.unsqueeze(0)).cpu() * exp_term / (torch.sqrt(self.to_var(det_cov).cpu()) + eps).unsqueeze(0),
                      dim=1) + eps)

        if size_average:
            sample_energy = torch.mean(sample_energy)

        return sample_energy, cov_diag

    def loss_function(self, x, x_hat, z, gamma, lambda_energy, lambda_cov_diag):
        recon_error = torch.mean((x.view(*x_hat.shape) - x_hat) ** 2)
        #print (z, gamma)
        phi, mu, cov = self.compute_gmm_params(z, gamma)
        
        #print (z, phi, mu, cov)
        sample_energy, cov_diag = self.compute_energy(z, phi, mu, cov)
#         print ("recon_error",recon_error)
#         print ("lambda_energy",lambda_energy)
#         print ("lambda_cov_diag",lambda_cov_diag)
#         cov_diag = cov_diag.float()
#         print ("cov_diag",cov_diag)
        loss = recon_error + lambda_energy * sample_energy + lambda_cov_diag * cov_diag
        
        return loss, sample_energy, recon_error, cov_diag

/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", n

In [ ]:
detectors = DAGMM(num_epochs=10, sequence_length=3)
detectors.fit(X_train.iloc[:,:-1].copy())

score = detectors.predict(Feature.iloc[:,:-1].copy())
output = pd.DataFrame({"cano_locdt_index":Feature.iloc[:,-1]})
output["score"] = score

print (output.shape)

output["cosine_errors_mean"] = detectors.prediction_details["cosine_errors_mean"]
output["euclidean_errors_mean"]  = detectors.prediction_details["euclidean_errors_mean"]
data = detectors.prediction_details["reconstructions_mean"]
reconstructions_mean = pd.DataFrame(data.T,
             columns = ["reconstructions_mean_latent_features_{}".format(i) for i in range(data.shape[0])]
            )
print (reconstructions_mean.shape)
data = detectors.prediction_details["latent_representations"]
latent_representations = pd.DataFrame(data.T,
             columns = ["latent_representations_latent_features_{}".format(i) for i in range(data.shape[0])]
            )
print (latent_representations.shape)
output = pd.concat([output,reconstructions_mean,latent_representations], axis = 1)
print (output.shape)

feature = []
for i in range(len(output)):
    if i%3 == 2:
        feature.append(output.iloc[i:i+1])
feature = pd.concat(feature,axis = 0)

feature.to_csv("/data/yunrui_li/fraud/fraud_detection/features/DAGMM_features_modified.csv", index = False)


 90%|█████████ | 9/10 [3:23:38<22:44, 1364.61s/it]  

In [ ]:
output.iloc[35:135]

In [ ]:
feature.head(100)

In [ ]:
df_train[df_train.cano == 2].sort_values(by = ["cano","locdt"])

In [19]:
import pandas as pd
import sys
sys.path.append("../fraud_detection/src/")
from util import s_to_time_format, string_to_datetime,hour_to_range

feature = pd.read_csv("/data/yunrui_li/fraud/fraud_detection/features/DAGMM_features_less_input.csv")


df_train = pd.read_csv("/data/yunrui_li/fraud/dataset/train.csv")
df_test = pd.read_csv("/data/yunrui_li/fraud/dataset/test.csv")

for df in [df_train, df_test]:
    # pre-processing
    df["loctm_"] = df.loctm.astype(int).astype(str)
    df.loctm_ = df.loctm_.apply(s_to_time_format).apply(string_to_datetime)
    # time-related feature
    df["loctm_hour_of_day"] = df.loctm_.apply(lambda x: x.hour)
    df["loctm_minute_of_hour"] = df.loctm_.apply(lambda x: x.minute)
    df["loctm_second_of_min"] = df.loctm_.apply(lambda x: x.second)

    # removed the columns no need
    #df.drop(columns = ["loctm_", "loctm","txkey"], axis = 1, inplace = True)

df_train["cano_locdt_index"] = ["{}_{}_{}_{}_{}".format(str(i),str(j),str(k),str(l),str(m)) for i,j,k,l,m in zip(df_train.cano,
                                                                                   df_train.locdt,
                                                                                   df_train.loctm_hour_of_day,
                                                                                   df_train.loctm_minute_of_hour,
                                                                                   df_train.loctm_second_of_min,
                                                                                  )]
df_test["cano_locdt_index"] = ["{}_{}_{}_{}_{}".format(str(i),str(j),str(k),str(l),str(m)) for i,j,k,l,m in zip(df_test.cano,
                                                                                  df_test.locdt,
                                                                                  df_test.loctm_hour_of_day,
                                                                                  df_test.loctm_minute_of_hour,
                                                                                  df_test.loctm_second_of_min,
                                                                                 )]

print ("df_train", df_train.shape)
print ("df_test", df_test.shape)

df_train_ = df_train.merge(feature, on = "cano_locdt_index", how = "left").drop_duplicates("txkey")
df_test_ = df_test.merge(feature, on = "cano_locdt_index", how = "left").drop_duplicates("txkey")

# df_train_ = df_train.merge(df, on = "cano_locdt_index", how = "left").drop_duplicates("txkey")
# df_test_ = df_test.merge(df, on = "cano_locdt_index", how = "left").drop_duplicates("txkey")
# # df_train_ = df_train.merge(df, on = "cano_locdt_index", how = "left")\
# # .drop_duplicates(subset =["cano","locdt","cosine_errors_mean","euclidean_errors_mean","reconstructions_mean_latent_features_0","score"])
# # df_test_ = df_test.merge(df, on = "cano_locdt_index", how = "left")\
# # .drop_duplicates(subset =["cano","locdt","cosine_errors_mean","euclidean_errors_mean","reconstructions_mean_latent_features_0","score"])


df_train (1521787, 28)
df_test (421665, 27)


In [20]:
feature

,cano_locdt_index,score,cosine_errors_mean,euclidean_errors_mean,reconstructions_mean_latent_features_0,reconstructions_mean_latent_features_1,reconstructions_mean_latent_features_2,reconstructions_mean_latent_features_3,reconstructions_mean_latent_features_4,reconstructions_mean_latent_features_5,latent_representations_latent_features_0,latent_representations_latent_features_1,latent_representations_latent_features_2,latent_representations_latent_features_3,latent_representations_latent_features_4
0,0_1_15_19_48,-22.662193,-1.084394,-1.117315,-0.296740,-0.068038,-0.053378,-0.296283,-0.163238,-0.179484,0.894681,-1.117315,-1.084394,-4.337092,4.635670
1,0_4_15_44_7,-27.490714,-4.578724,-3.045611,0.166659,0.176413,0.220550,0.435034,0.350870,0.346924,4.344673,-3.045611,-4.578724,-1.159269,1.482810
2,0_20_14_53_42,-27.474175,-5.092361,-3.328975,0.110268,0.328782,0.303428,0.583366,0.454730,0.445215,4.851853,-3.328975,-5.092361,-0.691998,1.019327
3,0_29_15_22_43,-27.213982,-5.102502,-3.334569,0.142504,0.339089,0.321281,0.601988,0.465076,0.455646,4.861865,-3.334569,-5.102502,-0.682776,1.010175
4,0_37_14_37_10,-26.059532,-5.084417,-3.324591,0.110154,0.350612,0.290773,0.569704,0.444154,0.436974,4.844006,-3.324591,-5.084417,-0.699225,1.026497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1943447,213570_119_13_56_3,-14.883758,-0.437768,-0.760364,-0.615070,-0.201883,-0.147716,-0.450633,-0.234637,-0.249322,0.256390,-0.760364,-0.437768,-4.925027,5.219095
1943448,213571_119_21_46_3,-14.677338,-2.131566,-1.695157,-0.354073,-0.043709,0.025809,-0.096460,0.043120,0.011579,1.928665,-1.695157,-2.131566,-3.384656,3.690858
1943449,213571_119_21_46_22,-20.515411,-3.918107,-2.680963,-0.278236,0.270400,0.166815,0.369971,0.299933,0.302892,3.692516,-2.680963,-3.918107,-1.759879,2.078836
1943450,213572_120_14_14_8,-16.658675,0.084298,-0.472286,-0.216788,-0.266204,-0.137874,-0.480453,-0.304188,-0.309235,-0.259116,-0.472286,0.084298,-5.399842,5.690127


In [21]:
df_train_.shape,df_train.shape

((1521787, 42), (1521787, 28))

In [22]:
df_test_.shape,df_test.shape

((421665, 41), (421665, 27))

In [14]:
df_test_

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,reconstructions_mean_latent_features_1,reconstructions_mean_latent_features_2,reconstructions_mean_latent_features_3,reconstructions_mean_latent_features_4,reconstructions_mean_latent_features_5,latent_representations_latent_features_0,latent_representations_latent_features_1,latent_representations_latent_features_2,latent_representations_latent_features_3,latent_representations_latent_features_4
0,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.357971,0.336542,0.619472,0.475497,0.465823,4.869079,-3.338600,-5.109807,-0.676131,1.003581
1,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.358617,0.336270,0.619228,0.475304,0.465682,4.868975,-3.338541,-5.109702,-0.676227,1.003676
2,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.354448,0.326729,0.608709,0.468585,0.459539,4.864737,-3.336174,-5.105409,-0.680131,1.007551
3,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.356239,0.333001,0.615862,0.473416,0.463803,4.867579,-3.337761,-5.108288,-0.677512,1.004953
4,6881,163188,116799,513.80,5,0,N,0,N,N,...,0.356255,0.331535,0.614534,0.472718,0.463102,4.866936,-3.337402,-5.107636,-0.678105,1.005541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433252,5975,147326,79511,633.76,5,62,N,2,N,N,...,-0.268523,-0.136616,-0.480806,-0.304450,-0.309710,-0.264438,-0.469330,0.089669,-5.404742,5.694973
433253,6716,14305,136493,952.84,5,62,N,5,N,N,...,-0.174444,-0.120915,-0.381662,-0.164746,-0.181103,0.457147,-0.872635,-0.641159,-4.740130,5.035600
433254,5975,156543,137963,713.42,5,62,N,4,N,N,...,-0.112124,-0.065254,-0.250773,-0.075432,-0.090175,0.765434,-1.044980,-0.953386,-4.456158,4.753905
433255,6231,156543,137964,903.94,5,62,N,5,N,N,...,-0.136906,-0.095623,-0.316338,-0.118455,-0.135043,0.611447,-0.958913,-0.797441,-4.598007,4.894596


In [16]:
df_test_[df_test_.cano == 141087]

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,cano_locdt_index,score,cosine_errors_mean,euclidean_errors_mean,reconstructions_mean_latent_features_0,reconstructions_mean_latent_features_1,reconstructions_mean_latent_features_2,reconstructions_mean_latent_features_3,reconstructions_mean_latent_features_4,reconstructions_mean_latent_features_5,latent_representations_latent_features_0,latent_representations_latent_features_1,latent_representations_latent_features_2,latent_representations_latent_features_3,latent_representations_latent_features_4
14328,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-14.252995,-1.679052,-1.445428,-0.209157,-0.055424,-0.029221,-0.233480,-0.111538,-0.136566,1.481778,-1.445428,-1.679052,-3.796282,4.099112
14329,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-22.822488,-4.961403,-3.256729,0.151077,0.199574,0.295812,0.540599,0.426344,0.416311,4.722487,-3.256729,-4.961403,-0.811152,1.137516
14330,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-24.854324,-5.107556,-3.337358,0.161351,0.355327,0.331344,0.613949,0.472143,0.462662,4.866856,-3.337358,-5.107556,-0.678179,1.005613
14331,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-24.854324,-5.107556,-3.337358,0.161351,0.355327,0.331344,0.613949,0.472143,0.462662,4.866856,-3.337358,-5.107556,-0.678179,1.005613
14335,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-14.252995,-1.679052,-1.445428,-0.209157,-0.055424,-0.029221,-0.233480,-0.111538,-0.136566,1.481778,-1.445428,-1.679052,-3.796282,4.099112
14336,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-22.822488,-4.961403,-3.256729,0.151077,0.199574,0.295812,0.540599,0.426344,0.416311,4.722487,-3.256729,-4.961403,-0.811152,1.137516
14337,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-24.854324,-5.107556,-3.337358,0.161351,0.355327,0.331344,0.613949,0.472143,0.462662,4.866856,-3.337358,-5.107556,-0.678179,1.005613
14338,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-24.854324,-5.107556,-3.337358,0.161351,0.355327,0.331344,0.613949,0.472143,0.462662,4.866856,-3.337358,-5.107556,-0.678179,1.005613
14339,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-14.252995,-1.679052,-1.445428,-0.209157,-0.055424,-0.029221,-0.233480,-0.111538,-0.136566,1.481778,-1.445428,-1.679052,-3.796282,4.099112
14340,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15,-22.822488,-4.961403,-3.256729,0.151077,0.199574,0.295812,0.540599,0.426344,0.416311,4.722487,-3.256729,-4.961403,-0.811152,1.137516


In [17]:
df_test[df_test.cano == 141087]

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,cano_locdt_index
13730,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15
13734,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15
13735,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15
13738,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,116,451,0,N,5817,102,0,10,19,20,141087_116_10_19_20
13741,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,108,451,0,N,5817,102,0,10,19,15,141087_108_10_19_15
13750,0,57136,141087,465.62,5,0,N,2,N,N,0,N,0,114,451,0,N,5817,102,0,10,18,49,141087_114_10_18_49


In [15]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

df_train_.sample(100)

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,cano_locdt_index,score,cosine_errors_mean,euclidean_errors_mean,reconstructions_mean_latent_features_0,reconstructions_mean_latent_features_1,reconstructions_mean_latent_features_2,reconstructions_mean_latent_features_3,reconstructions_mean_latent_features_4,reconstructions_mean_latent_features_5,latent_representations_latent_features_0,latent_representations_latent_features_1,latent_representations_latent_features_2,latent_representations_latent_features_3,latent_representations_latent_features_4
1338902,6557,25262,12785,831.19,5,62,N,4,N,N,0,5,N,0,90,274,31899,N,5817,102,0,15,28,17,12785_90_15_28_17,-24.174337,-3.292722,-2.335866,-0.082416,0.121702,0.121146,0.176969,0.165677,0.168553,3.075058,-2.335866,-3.292722,-2.328648,2.643121
994941,6881,154515,59034,513.80,5,0,N,0,N,N,0,5,N,0,36,457,68978,N,0,102,0,13,46,22,59034_36_13_46_22,-27.141203,-5.101823,-3.334194,0.141519,0.340639,0.319731,0.600463,0.464023,0.454771,4.861195,-3.334194,-5.101823,-0.683393,1.010788
1296720,6881,130336,81223,513.80,5,0,N,0,N,N,0,5,N,0,86,457,59369,N,0,102,0,8,42,46,81223_86_8_42_46,-27.262924,-5.109449,-3.338402,0.167880,0.357690,0.335696,0.618578,0.474941,0.465309,4.868725,-3.338402,-5.109449,-0.676457,1.003905
420241,6716,52310,99766,394.74,5,62,N,4,N,N,0,5,N,0,73,247,70286,N,5813,102,0,15,52,31,99766_73_15_52_31,-27.283774,-5.106021,-3.336511,0.152683,0.345276,0.328367,0.610124,0.470179,0.460430,4.865341,-3.336511,-5.106021,-0.679574,1.006999
772357,6716,149403,45252,2435.27,5,62,Y,2,N,N,0,5,Y,3,29,203,38840,N,5820,102,0,15,50,5,45252_29_15_50_5,-26.389828,-5.102540,-3.334590,0.144512,0.342666,0.321152,0.602018,0.464862,0.455648,4.861905,-3.334590,-5.102540,-0.682739,1.010139
1343567,6189,3162,31185,1017.48,5,62,Y,2,N,N,0,5,N,0,8,280,90223,N,1170,102,0,13,33,14,31185_8_13_33_14,-26.369760,-5.100560,-3.333498,0.151044,0.359864,0.317667,0.598848,0.461655,0.453692,4.859947,-3.333498,-5.100560,-0.684543,1.011927
1180465,6677,159134,92060,677.88,5,62,N,5,N,N,0,5,N,0,81,255,25660,N,6022,102,0,13,26,4,92060_81_13_26_4,-27.330098,-5.106344,-3.336689,0.160970,0.357589,0.328726,0.610994,0.469881,0.460856,4.865659,-3.336689,-5.106344,-0.679281,1.006707
871453,6716,134678,147519,719.26,5,62,Y,2,N,Y,0,5,N,0,16,306,34176,N,5820,102,0,17,3,30,147519_16_17_3_30,-27.036632,-5.009473,-3.283254,0.038854,0.281649,0.244271,0.501843,0.402305,0.395506,4.769998,-3.283254,-5.009473,-0.767403,1.094127
818695,6834,64586,172472,680.57,5,62,N,5,N,N,0,5,N,0,76,263,97361,N,5817,102,0,21,21,59,172472_76_21_21_59,-26.334770,-5.093943,-3.329848,0.125013,0.351747,0.310570,0.592572,0.460290,0.450709,4.853412,-3.329848,-5.093943,-0.690563,1.017899
623723,6767,95586,117474,794.93,5,62,N,5,N,N,0,5,N,0,59,277,18344,N,5817,102,0,17,8,37,117474_59_17_8_37,-27.305336,-5.108401,-3.337824,0.165613,0.358213,0.333240,0.615990,0.473210,0.463802,4.867691,-3.337824,-5.108401,-0.677410,1.004850


In [ ]:
df_test_.sample(100)

In [ ]:
df_test_[df_test_.cano == 73737]

In [ ]:
df[df.cano_locdt_index == "73737_116"]

In [ ]:
output[output.cano_locdt_index == "73737_116"]

In [ ]:
output[output.cano_locdt_index.str.contains("73737",na = False)]

In [ ]:
df_test_[df_test_.cano == 73737].sort_values(by = ["cano","locdt"])

In [ ]:
df_train_.shape, df_train.shape

In [ ]:
df_test_.shape, df_test.shape

In [ ]:
df_test[df_test.cano == 73737].sort_values(by = ["cano","locdt"])

In [ ]:
df_test_[df_test_.cano == 73737].sort_values(by = ["cano","locdt"]).drop_duplicates(subset =["cano","locdt","cosine_errors_mean","score"])

In [ ]:
df[df.cano_locdt_index.str.contains("73737")]